In [18]:
from pytrends.request import TrendReq
import pandas as pd
from pandas import ExcelWriter
import time
from random import randint
import pytrends

In [2]:
username = 'ebao@toolsgroup.com'
password = 'blx112358'
pytrend = TrendReq(username,password)

keywords = {
            'ray-ban':['ray ban','ray-ban','ray ban sunglasses','ray ban wayfarer'],
            'ray-ban_vista':['ray ban vista','ray ban glasses','ray ban eyeglasses','ray ban frame','ray ban glasses frame'],
            'oakley':['oakley','oakley sunglasses','oakley goggles'],
            'oakley_vista':['oakley vista','oakley glasses','oakley eyeglasses','oakley frame','oakley glasses frame']
           }

geocodes = {
            'TURKEY':'Turkey',
            'INDIA':'India',
            'BRASIL':'Brazil',
            'JAPAN':'Japan',
            'KOREA':'Korea, Republic of',
            'ASTRL':'Australia',
            'MEXICO':'Mexico',
            'PHIL':'Philippines',
            'NORAME':['United States','Canada'],
            'CHI_HK':'Hong Kong'
           }

isofile = 'data/ISO 3166-1-alpha-2.csv'


In [3]:
def get_geo(geodict,isofile):
    """Load ISO country code file, given geocodes dict return a list of country codes"""
    iso = pd.read_csv(isofile)
    geolist = []
    
    for code,isoname in geodict.items():
    
        if type(isoname) == list:
            geolist.append([(iso.loc[iso['Name'] == c, 'Code'].values[0],code) for c in isoname])
        else:
            geolist.append((iso.loc[iso['Name'] == isoname, 'Code'].values[0],code))
            
    return geolist

In [4]:
def get_trend(brand,date=None,geo=None,hl=None,p1=5,p2=10):
    """
    Given Brand name return monthly google trend data (time > 36m), default time 47m. Loop predifeined keywords automaticlly.
    """
    global keywords
    
    if type(geo) != list:
        for i,w in enumerate(keywords[brand]):
            payload = {'q':w,
                       'date':'1/2013 47m', # 1/2013,14,15,10/16
                       'geo':None,
                       'hl':None
                      }
            if date != None:
                payload['date'] = date
            if geo != None:
                payload['geo'] = geo
            if hl != None:
                payload['hl'] = hl
            print(payload)
            try:
                if i == 0:
                    trend = pytrend.trend(payload, return_type='dataframe')
                    time.sleep(randint(p1,p2))
                if i > 0:
                    try:
                        trend[w+'_'+geo] = pytrend.trend(payload, return_type='dataframe')
                        time.sleep(randint(p1,p2))
                    except UnboundLocalError:
                        trend = pytrend.trend(payload, return_type='dataframe')
                        time.sleep(randint(p1,p2))
            except IndexError as e:
                print('%s on keyword: %s, resuming...' %(e,w))
                time.sleep(randint(p1,p2))
                pass
        try:
            trend['mean'] = trend.mean(axis=1)
        except UnboundLocalError:
            return None
        return trend
    else:
        for idx,g in enumerate(geo):
            if idx == 0:
                for i,w in enumerate(keywords[brand]):
                    payload = {'q':w,
                               'date':'1/2013 47m', # 1/2013,14,15,10/16
                               'geo':g,
                               'hl':None
                              }
                    if date != None:
                        payload['date'] = date
                    if hl != None:
                        payload['hl'] = hl
                    print(payload)
                    try:
                        if i == 0:
                            trend = pytrend.trend(payload, return_type='dataframe')
                            time.sleep(randint(p1,p2))
                        if i > 0:
                            try:
                                trend[w+'_'+geo[idx]] = pytrend.trend(payload, return_type='dataframe')
                                time.sleep(randint(p1,p2))
                            except UnboundLocalError:
                                trend = pytrend.trend(payload, return_type='dataframe')
                                time.sleep(randint(p1,p2))
                    except IndexError as e:
                        print('%s on keyword: %s, resuming...' %(e,w))
                        time.sleep(randint(p1,p2))
                        pass
            if idx > 0:
                for i,w in enumerate(keywords[brand]):
                    payload = {'q':w,
                               'date':'1/2013 47m', # 1/2013,14,15,10/16
                               'geo':g,
                               'hl':None
                              }
                    if date != None:
                        payload['date'] = date
                    if hl != None:
                        payload['hl'] = hl
                    print(payload)
                    try:
                        trend[w+'_'+geo[idx]] = pytrend.trend(payload, return_type='dataframe')
                        time.sleep(randint(p1,p2))
                    except IndexError as e:
                            print('%s on keyword: %s, resuming...' %(e,w))
                            time.sleep(randint(p1,p2))
                            pass
        try:
            trend['mean'] = trend.mean(axis=1)
        except UnboundLocalError:
            return None
        return trend

In [25]:
def save_xls(list_dfs, xls_path):
    writer = ExcelWriter(xls_path)
    for (df,luxcode) in list_dfs:
        try:
            df.to_excel(writer,'%s' %(luxcode))
        except AttributeError:
            pass
            
    writer.save()

In [5]:
def get_numbers(keywordsdict,geodict):
    """
    Calculate howlong to pause between each query, Google Rate Limit around 10/min
    """
    n_k = 0
    n_g = 0
    for k,v in keywordsdict.items():
        n_k += len(v)
    for k,v in geodict.items():
        if type(v) == list:
            n_g += len(v)
        else:
            n_g += 1
    time = n_k*n_g/10
    
    print('No. of keywords: %i\tNo. of countries: %i\nTotal query: %i\tTime needed: %4.1fm' %(n_k,n_g,n_k*n_g,time))

In [36]:
dflist = []
brand = 'oakley_vista'

In [37]:
def geotrends(brand,geolist,p1,p2):
    
    global dflist
    
    for code in geolist:
        if type(code) != list:
            isocode,luxcode = code
            print(isocode,luxcode)
            trend = get_trend(brand,geo=isocode)
            dflist.append((trend,luxcode))
            time.sleep(randint(p1,p2))
        else:
            isocode = [x for (x,y) in code]
            luxcode = list(set([y for (x,y) in code]))[0]
            print(' '.join(isocode) + ' ' + luxcode)
            trend = get_trend(brand,geo=isocode)
            dflist.append((trend,luxcode))
            time.sleep(randint(p1,p2))

In [38]:
while True:
    try:
        geotrends(brand,geos[len(dflist):],5,10)
        break
    except:
        print('ERROR PAUSING 5m')
        time.sleep(60*5)
        
save_xls(dflist,'data/Regional_Trend_Oakley_vista.xlsx')

IN INDIA
{'q': 'oakley vista', 'date': '1/2013 47m', 'hl': None, 'geo': 'IN'}
list index out of range on keyword: oakley vista, resuming...
{'q': 'oakley glasses', 'date': '1/2013 47m', 'hl': None, 'geo': 'IN'}
{'q': 'oakley eyeglasses', 'date': '1/2013 47m', 'hl': None, 'geo': 'IN'}
list index out of range on keyword: oakley eyeglasses, resuming...
{'q': 'oakley frame', 'date': '1/2013 47m', 'hl': None, 'geo': 'IN'}
list index out of range on keyword: oakley frame, resuming...
{'q': 'oakley glasses frame', 'date': '1/2013 47m', 'hl': None, 'geo': 'IN'}
list index out of range on keyword: oakley glasses frame, resuming...
PH PHIL
{'q': 'oakley vista', 'date': '1/2013 47m', 'hl': None, 'geo': 'PH'}
list index out of range on keyword: oakley vista, resuming...
{'q': 'oakley glasses', 'date': '1/2013 47m', 'hl': None, 'geo': 'PH'}
{'q': 'oakley eyeglasses', 'date': '1/2013 47m', 'hl': None, 'geo': 'PH'}
{'q': 'oakley frame', 'date': '1/2013 47m', 'hl': None, 'geo': 'PH'}
list index out of

In [6]:
get_numbers(keywords,geocodes)

No. of keywords: 17	No. of countries: 11
Total query: 187	Time needed: 18.7m


In [7]:
geos = get_geo(geocodes,isofile)
geos

[('IN', 'INDIA'),
 ('PH', 'PHIL'),
 ('MX', 'MEXICO'),
 ('HK', 'CHI_HK'),
 ('KR', 'KOREA'),
 ('TR', 'TURKEY'),
 ('BR', 'BRASIL'),
 [('US', 'NORAME'), ('CA', 'NORAME')],
 ('JP', 'JAPAN'),
 ('AU', 'ASTRL')]

In [32]:
keywords

{'oakley': ['oakley', 'oakley sunglasses', 'oakley goggles'],
 'oakley_vista': ['oakley vista',
  'oakley glasses',
  'oakley eyeglasses',
  'oakley frame',
  'oakley glasses frame'],
 'ray-ban': ['ray ban', 'ray-ban', 'ray ban sunglasses', 'ray ban wayfarer'],
 'ray-ban_vista': ['ray ban vista',
  'ray ban glasses',
  'ray ban eyeglasses',
  'ray ban frame',
  'ray ban glasses frame']}

In [39]:
for x,y in dflist:
    if x is None:
        print(y)

CHI_HK
KOREA
TURKEY
BRASIL
JAPAN


In [64]:
brand = 'ray-ban_vista'
p1,p2 = 5,10

for code in geos[9:]:
    if type(code) != list:
        isocode,luxcode = code
        print(isocode,luxcode)
        trend = get_trend(brand,geo=isocode)
        dflist.append((trend,luxcode))
        time.sleep(randint(p1,p2))   
    else:
        isocode = [x for (x,y) in code]
        luxcode = list(set([y for (x,y) in code]))[0]
        print(' '.join(isocode) + ' ' + luxcode)
        trend = get_trend(brand,geo=isocode)
        dflist.append((trend,luxcode))
        time.sleep(randint(p1,p2))

KR KOREA
{'q': 'ray ban vista', 'hl': None, 'geo': 'KR', 'date': '1/2013 47m'}
list index out of range on keyword: ray ban vista, resuming...
{'q': 'ray ban glasses', 'hl': None, 'geo': 'KR', 'date': '1/2013 47m'}
list index out of range on keyword: ray ban glasses, resuming...
{'q': 'ray ban eyeglasses', 'hl': None, 'geo': 'KR', 'date': '1/2013 47m'}
list index out of range on keyword: ray ban eyeglasses, resuming...
{'q': 'ray ban frame', 'hl': None, 'geo': 'KR', 'date': '1/2013 47m'}
list index out of range on keyword: ray ban frame, resuming...
{'q': 'ray ban glasses frame', 'hl': None, 'geo': 'KR', 'date': '1/2013 47m'}
list index out of range on keyword: ray ban glasses frame, resuming...


In [66]:
for df,area in dflist:
    if df is None:
        print(df,area)

None JAPAN
None KOREA


In [51]:
dflist[-1][0]

,ray ban glasses,ray ban eyeglasses_PH,mean
Date,,,
2013-01-01,32,26,29.0
2013-02-01,57,45,51.0
2013-03-01,42,64,53.0
2013-04-01,100,55,77.5
2013-05-01,93,38,65.5
2013-06-01,10,25,17.5
2013-07-01,18,23,20.5
2013-08-01,28,79,53.5
2013-09-01,56,46,51.0


In [24]:
def save_xls(list_dfs, xls_path):
    writer = ExcelWriter(xls_path)
    for (df,luxcode) in list_dfs:
        try:
            df.to_excel(writer,'%s' %(luxcode))
        except AttributeError:
            pass
            
    writer.save()

In [71]:
save_xls(dflist,'data/Regional_Trend_Ray-Ban_vista.xlsx')